In [1]:
conda install selenium

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [74]:
import numpy as numpy
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import datetime
import time

def lazyloading(driver):
    # Get the current number of rows
    current_rows_number = len(driver.find_elements_by_class_name('explorerCard__explorerCard--3Q7_0'))
    total_number = int(driver.find_elements_by_css_selector('.querySummary__querySummary--39WP2')[0].text.split('Showing ')[1].split('Sparkling wines')[0])
    print("Total Number: ", total_number)
    while current_rows_number < total_number:
        # Scroll down to make new XHR (request more table rows)
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        try:
            driver.set_window_position(0,-1000)
            # Wait until number of rows increased
            time.sleep(10)
            wait(driver, 20).until(lambda drive: len(driver.find_elements_by_class_name('explorerCard__explorerCard--3Q7_0')) > current_rows_number)
            # Update variable with current rows number
            current_rows_number = len(driver.find_elements_by_class_name('explorerCard__explorerCard--3Q7_0'))
            print("Curr Row Number: ", current_rows_number)
        # If number of rows remains the same after 5 seconds passed, break the loop
        # as there no more rows to receive
        except TimeoutException:
            break

In [75]:
from wine_setup import WineSetup

sparkling_wine_url = "https://www.vivino.com/explore?e=eJwNizsKgDAQBW_z6ohYvs4biJWIrDGGgFHJxt_tTTNTDBMTK8Sw0yDKy8YY2I99B1vQ4izVr7wlBZdlwzEzSQ6710lul8Q7HFycWjx5GFmXWYsvhdLKD5MkH4Q="
sparkling_wine = Wine(sparkling_wine_url)
sparkling_wine.setup_driver()
sparkling_wine.run_timestamp()
lazyloading(sparkling_wine.driver)
sparkling_wine.run_timestamp()

Current Timestamp:  05-Feb-2020 (19:31:37.142036)
Total Number:  1195
Curr Row Number:  50
Curr Row Number:  75
Curr Row Number:  100
Curr Row Number:  125
Curr Row Number:  150
Curr Row Number:  175
Curr Row Number:  200
Curr Row Number:  225
Curr Row Number:  250
Curr Row Number:  275
Curr Row Number:  300
Curr Row Number:  325
Curr Row Number:  350
Curr Row Number:  375
Curr Row Number:  400
Curr Row Number:  425
Curr Row Number:  450
Curr Row Number:  475
Curr Row Number:  500
Curr Row Number:  525
Curr Row Number:  550
Curr Row Number:  575
Curr Row Number:  600
Curr Row Number:  625
Curr Row Number:  650
Curr Row Number:  675
Curr Row Number:  700
Curr Row Number:  725
Curr Row Number:  750
Curr Row Number:  775
Curr Row Number:  800
Curr Row Number:  825
Curr Row Number:  850
Curr Row Number:  875
Curr Row Number:  900
Curr Row Number:  925
Curr Row Number:  950
Curr Row Number:  975
Curr Row Number:  1000
Curr Row Number:  1025
Curr Row Number:  1050
Curr Row Number:  1075
Curr

In [76]:
sparkling_html = BeautifulSoup(sparkling_wine.driver.page_source, 'lxml')
sparkling_div = sparkling_html.find("div", {"class": "explorerPage__results--3wqLw"})
sparkling_rows = sparkling_html.find_all("div", {"class": "explorerCard__explorerCard--3Q7_0"})

In [77]:
import copy

# austria_wine_df = pd.DataFrame()
all_rows = []

# Let's store each row as a dictionary 
empty_row = {
    "title": None, "location": None, "country": None, "price": None, "type": None, "ratings": None, "num_ratings": None, "reviews": None, "url": None
}

for row in sparkling_rows:
    new_row = copy.copy(empty_row)
    # A list of all the entries in the row.
    new_row['title'] = row.find("span", {"class": "vintageTitle__wine--U7t9G"}).text
    location = row.find("div", {"class": "vintageLocation__vintageLocation--1DF0p"})
    new_row['location'] = location.findChildren()[-1].text
    country = row.find("i", {"class": "vintageLocation__countryFlag--1HbXr"})['title']
    new_row['country'] = country
    price_button = row.find("button", {"class": "addToCartButton__addToCartButton--qZv9F"})
    if price_button:
        new_row['price'] = (float(price_button.find("span").text.replace("$", "")))
    new_row['type'] = 'Sparkling'
    new_row['ratings'] = row.find("div", {"class": "vivinoRatingWide__averageValue--1zL_5"}).text
    new_row['num_ratings'] = int(row.find("div", {"class": "vivinoRatingWide__basedOn--s6y0t"}).text.split()[0])
    review_div = row.find("div", {"class": "review__note--2b2DB"})
    if review_div:
        new_row['reviews'] = review_div.text
    clean_div = row.find("div", {"class": "cleanWineCard__cleanWineCard--tzKxV cleanWineCard__row--CBPRR"})
    if clean_div:
        new_row['url'] = 'https://www.vivino.com' + clean_div.find("a")['href']
    all_rows.append(new_row)

In [78]:
all_rows

[{'title': 'J. Schram Brut Rosé 2009',
  'location': 'North Coast',
  'country': 'United States',
  'price': None,
  'type': 'Sparkling',
  'ratings': '4.7',
  'num_ratings': 96,
  'reviews': None,
  'url': 'https://www.vivino.com/schramsberg-vineyards-j-schram-brut-rose/w/4901?year=2009'},
 {'title': 'J. Schram Brut Rosé 2010',
  'location': 'North Coast',
  'country': 'United States',
  'price': None,
  'type': 'Sparkling',
  'ratings': '4.7',
  'num_ratings': 53,
  'reviews': None,
  'url': 'https://www.vivino.com/schramsberg-vineyards-j-schram-brut-rose/w/4901?year=2010'},
 {'title': 'Le Rêve Late Disgorged 2007',
  'location': 'Los Carneros',
  'country': 'United States',
  'price': None,
  'type': 'Sparkling',
  'ratings': '4.7',
  'num_ratings': 51,
  'reviews': None,
  'url': 'https://www.vivino.com/domaine-carneros-le-reve-late-disgorged/w/1882447?year=2007'},
 {'title': "Commander's Palace Brut 2013",
  'location': 'Green Valley of Russian River Valley',
  'country': 'United 

In [28]:
conda install pymongo

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [79]:
import pymongo
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
wine_db = client['wine_db']
wines = wine_db['wines']

In [80]:
for row in all_rows:
    wines.insert_one(row)